In [7]:
REPLICATE_API_TOKEN = "r8_fS0XoYUbgQyhYhhpNjIZfCF8I9oWWVV1BZTKk"
import replicate


In [1]:
gender = "f"
gender_clothing = {"m": "shirtless", "f": "wearing bikini", "default": "wearing long swim shorts"}

prompt1 ="headshot of smiling model posing with a lot of cute puppies wearing casual clothes posing for dating app headshot. indoor blurry background. the lighting is warm, possibly from a setting sun, creating a soft glow around him, enhancing the casual and relaxed vibe of the image. the setting seems to be outdoors, likely in an urban environment, with the blurred background hinting at a street or park-like area. this image likely portrays a youthful, active, and approachable individual, possibly in a lifestyle or fashion-related context."
prompt2 = "A model wearing a cozy Christmas sweater, standing by the fireplace with stockings and holiday decorations."
prompt3 = "pov photo of model seated at restaurant table across from camera, in romantic upscale setting facing camera. medium rare steak is on the table sliced into several pieces, on a wooden board, which also has a small dish of what appears to be a side condiment or salsa with chopped vegetables."
prompt4 = "professional headshot of smiling model wearing professional clothes posing for headshot. blurry indoor office background. The overall vibe of the image is one of professionalism, likely intended for a formal or business-related setting, such as a corporate headshot or a professional profile picture."
prompt5 = "TedX speaker model holding microphone with lanyard around his neck"
prompt6 = "beautiful influencer instagram model wearing elegant clothes sitting in private jet cabin, with leather interior, luxurious. champagne is on the table. outside is clouds because we are flying."
prompt7 = "model as fashion model in fashion shoot on catwalk."
prompt8 = "model as fashion model in street style shoot with diverse outfits."
prompt9 = f"the photo shows a fit, {gender_clothing.get(gender, gender_clothing['default'])} happy model on the beach, playing volleyball, seemingly in preparation for a serve. model appears focused, with their gaze fixed on the ball. the background includes other beachgoers and beach equipment, but they are slightly blurred, emphasizing the model as the focal point. the model has a muscular build, with defined arms, chest, and abs. the volleyball holding is a mikasa brand, commonly used in beach volleyball. the setting suggests a warm, sunny day, perfect for beach activities."
prompt10 = "model wearing casual clothes in polaroid classic photograph posing for photo indoors"
prompt11 = "casual profile headshot photo of model for Twitter. hasselblad photography."




SyntaxError: EOL while scanning string literal (1631666782.py, line 5)

In [9]:
replicate = replicate.Client(api_token=REPLICATE_API_TOKEN)  # Initialize the client with the token

In [10]:
model_id = "i-aayush/my-flux:9010d504cf84e942e2fdd0d5f08b8fa45a63905407e66ab4b036df51a2460569"
trigger_word = "aayu"
prompt1 =f"headshot of smiling {trigger_word} posing with a lot of cute puppies wearing casual clothes posing for dating app headshot. indoor blurry background. the lighting is warm, possibly from a setting sun, creating a soft glow around him, enhancing the casual and relaxed vibe of the image. the setting seems to be outdoors, likely in an urban environment, with the blurred background hinting at a street or park-like area. this image likely portrays a youthful, active, and approachable individual, possibly in a lifestyle or fashion-related context."
output = replicate.run(
    model_id,
    input={
        "model": "dev",
        "prompt": prompt1,
        "go_fast": False,
        "lora_scale": 1,
        "megapixels": "1",
        "num_outputs": 2,
        "aspect_ratio": "1:1",
        "output_format": "png",
        "guidance_scale": 3,
        "output_quality": 100,
        "prompt_strength": 0.96,
        "extra_lora_scale": 1,
        "num_inference_steps": 41
    }
)
print(output)

[<replicate.helpers.FileOutput object at 0x106114a90>, <replicate.helpers.FileOutput object at 0x105215d30>]


In [13]:
print(output)

[<replicate.helpers.FileOutput object at 0x106114a90>, <replicate.helpers.FileOutput object at 0x105215d30>]


In [14]:
# Assuming 'output' contains the list of FileOutput objects
for i, file_output in enumerate(output):
    file_url = file_output.url  # This attribute typically contains the URL
    print(f"Output {i + 1}: {file_url}")  # Print the URL of each output file

Output 1: https://replicate.delivery/xezq/rZvwtSR7NsINK9ooEf8m47brPK80SbtsqQQhe6fAkDKvcODoA/out-0.png
Output 2: https://replicate.delivery/xezq/TxNiQ7jbPjYvMNn1XCdSVyFafjj5VoFtUySrgf6Tq87XOnBUA/out-1.png
